In [2]:
import pandas as pd

# Load historical sales data from Excel files
df_2011 = pd.read_excel('2011.xlsx')
df_2012 = pd.read_excel('2012.xlsx')
df_2013 = pd.read_excel('2013.xlsx')
sales_current = pd.read_excel('2014.xlsx')

# Append historical sales data into a single DataFrame
sales_historical = pd.concat([df_2011, df_2012, df_2013], ignore_index=True)

# Combine parts of Order ID in current sales data
sales_current['Order ID'] = sales_current['Order ID Part 1'].astype(str) + '-' + sales_current['Order ID Part 2'].astype(str) + '-' + sales_current['Order ID Part 3'].astype(str)
sales_current.drop(['Order ID Part 1', 'Order ID Part 2', 'Order ID Part 3'], axis=1, inplace=True)

# Split 'City State Country' into separate columns
sales_current[['City', 'State', 'Country']] = sales_current['City State Country'].str.split(', ', expand=True)
sales_current.drop('City State Country', axis=1, inplace=True)

# Append current and historical sales data
fact_sales = pd.concat([sales_historical, sales_current], ignore_index=True)

# Add Order Priority column based on sales amount
def assign_priority(sales):
    if sales < 1000:
        return "Low"
    elif sales < 2500:
        return "Medium"
    elif sales < 5000:
        return "High"
    else:
        return "Critical"
fact_sales['Order Priority'] = fact_sales['Sales'].apply(assign_priority)

# Load category data and adjust the format
dim_category = pd.read_excel('Category.xlsx')
dim_category['Category'] = dim_category['Category'].ffill()
dim_category['Category'] = dim_category['Category'].str.title()
dim_category['Sub-Category'] = dim_category['Sub-Category'].str.title()

# Merge category data with sales data
merged_data = pd.merge(fact_sales, dim_category, on='Sub-Category', how='left')

# Load and process discount data
dim_discount = pd.read_excel('Discount.xlsx', skiprows=3)
dim_discount.columns = dim_discount.iloc[0]
dim_discount = dim_discount.drop(dim_discount.index[0])
dim_discount_melted = pd.melt(dim_discount, id_vars=['Order Priority'], var_name='Market', value_name='Discount')

# Merge discount data with sales data
final_merged_data = pd.merge(merged_data, dim_discount_melted, on=['Market', 'Order Priority'], how='left')

# Save the final merged DataFrame to a CSV file
final_merged_data.to_csv('Fact_Sales_With_Discounts.csv', index=False)
